In [2]:
%load_ext autoreload

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from sklearn.utils import shuffle
from torchsample.initializers import Uniform
from torchsample.modules import ModuleTrainer
from torchsample.metrics import CategoricalAccuracy

%aimport torchsample.modules

%matplotlib inline

/Users/rodrigo/Libs/torchsample/torchsample/datasets.py:16: UserWarning: Cant import nibabel.. Cant load brain images
  warnings.warn('Cant import nibabel.. Cant load brain images')


In [3]:
use_cuda = False
batch_size = 64

# Setup data

We're going to look at the IMDB dataset, which contains movie reviews from IMDB, along with their sentiment. Keras comes with some helpers for this dataset.

In [4]:
from keras.datasets import imdb
idx = imdb.get_word_index()

Using Theano backend.


This is the word list:

In [5]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

...and this is the mapping from id to word

In [6]:
idx2word = {v: k for k, v in idx.items()}

We download the reviews using code copied from keras.datasets:

In [7]:
from keras.utils.data_utils import get_file
import pickle

path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [8]:
len(x_train)

25000

Here's the 1st review. As you see, the words have been replaced by ids. The ids can be looked up in idx2word.

In [8]:
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

The first word of the first review is 23022. Let's see what that is.

In [9]:
idx2word[23022]

'bromwell'

Here's the whole review, mapped from ids to words.

In [10]:
' '.join([idx2word[o] for o in x_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

The labels are 1 for positive, 0 for negative.

In [9]:
labels_train_tensor = torch.from_numpy(np.array(labels_train))
labels_test_tensor = torch.from_numpy(np.array(labels_test))
labels_train[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Reduce vocab size by setting rare words to max index.

In [10]:
vocab_size = 5000

trn = [np.array([i if i < vocab_size - 1 else vocab_size - 1 for i in s]) for s in x_train]
test = [np.array([i if i < vocab_size - 1 else vocab_size - 1 for i in s]) for s in x_test]

Look at distribution of lengths of sentences.

In [11]:
lens = np.array(list(map(len, trn)))
(lens.max(), lens.min(), lens.mean())

(2493, 10, 237.71364)

Pad (with zero) or truncate each sentence to make consistent length.

In [12]:
seq_len = 500

from keras.preprocessing import sequence

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

trn_tensor = torch.from_numpy(trn).long()
test_tensor = torch.from_numpy(test).long()

This results in nice rectangular matrices that can be passed to ML algorithms. Reviews shorter than 500 words are pre-padded with zeros, those greater are truncated.

In [15]:
trn_tensor.size()

torch.Size([25000, 500])

## Create simple models

### Single hidden layer NN

The simplest model that tends to give reasonable results is a single hidden layer net. So let's try that. Note that we can't expect to get any useful results by feeding word ids directly into a neural net - so instead we use an embedding to replace them with a vector of 32 (initially random) floats for each word in the vocab.

In [16]:
import torch.nn as nn
import torch.nn.functional as F

class SingleHiddenLayerModule(nn.Module):
    def __init__(self):
        super().__init__()
        num_dimensions = 32
        self.embedding = nn.Embedding(vocab_size, num_dimensions)
        self.fc1 = nn.Linear(seq_len * num_dimensions, 100)
        self.dropout = nn.Dropout(0.7)
        self.fc2 = nn.Linear(100, 2)
        self.init()

    def forward(self, words_ids):
        x = self.embedding(words_ids) # x => torch.Size([64, 500, 32])
        x = x.view(x.size(0), -1) # x => torch.Size([64, 16000])
        x = self.fc1(x)
        x = F.relu(x, True)
        x = self.dropout(x)
        x = self.fc2(x)
        # result = F.sigmoid(x)
        result = x
        return result
    
    def init(self):
        torch.nn.init.constant(self.fc1.bias, val=0.0)
        torch.nn.init.constant(self.fc2.bias, val=0.0)

In [17]:
%autoreload 2

# criterion = nn.BCELoss()
criterion = nn.CrossEntropyLoss()
model = SingleHiddenLayerModule()
if(use_cuda):
    model.cuda()
    criterion.cuda()
trainer = ModuleTrainer(model)
trainer.set_optimizer(optim.Adam, lr=1e-3)
trainer.set_loss(criterion)
trainer.set_initializers([Uniform(module_filter="embedding*", a=-0.05, b=0.05), XavierUniform(module_filter="fc*")])
trainer.set_metrics([CategoricalAccuracy()])

# trainer.summary((trn_tensor.size(0), labels_train_tensor.size(0)))
model

SingleHiddenLayerModule (
  (embedding): Embedding(5000, 32)
  (fc1): Linear (16000 -> 100)
  (dropout): Dropout (p = 0.7)
  (fc2): Linear (100 -> 2)
)

In [18]:
trainer.fit(trn_tensor, labels_train_tensor, validation_data=(test_tensor, labels_test_tensor), 
            nb_epoch=2, batch_size=batch_size, shuffle=True)

Epoch 1/2: 392 batches [01:19,  2.72s/ batches, val_acc=81.47, val_loss=0.2871, acc=75.19, loss=0.4612]
Epoch 2/2: 392 batches [01:40,  6.92s/ batches, val_acc=89.91, val_loss=0.3095, acc=92.56, loss=0.1991]


The [stanford paper](http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf) that this dataset is from cites a state of the art accuracy (without unlabelled data) of 0.883. ~~So we're short of that, but on the right track.~~ We've already beaten the state of the art in 2011 with a simple Neural Net.

### Single conv layer with max pooling

A CNN is likely to work better, since it's designed to take advantage of ordered data. We'll need to use a 1D CNN, since a sequence of words is 1D.

In [48]:
import torch.nn as nn
import torch.nn.functional as F

class CnnMaxPoolingModule(nn.Module):
    def __init__(self):
        super().__init__()
        num_dimensions = 32
        self.embedding = nn.Embedding(vocab_size, num_dimensions)
        self.drop1 = nn.Dropout(0.2)
        self.conv1 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, padding=2, groups=1)
        self.fc1 = nn.Linear(seq_len * num_dimensions, 100)
        self.dropout = nn.Dropout(0.7)
        self.fc2 = nn.Linear(100, 2)
        self.init()

    def forward(self, words_ids):
        x = self.embedding(words_ids)  # x => torch.Size([B, 500, 32])
        x = x.permute(0, 2, 1)
        # print('emb', x.size())
        x = self.drop1(x) # x => torch.Size([B, 500, 32])
        x = self.conv1(x)   # x => torch.Size([B, 500, 64])
        x = F.relu(x, True)
        # print('conv1', x.size())
        x = self.drop1(x)   # x => torch.Size([B, 500, 64])
        x = F.max_pool1d(x, kernel_size=2)
        # print('max', x.size())
        
        x = x.view(x.size(0), -1)
        # print(x.size())
        
        x = self.fc1(x)
        x = F.relu(x, True)
        x = self.dropout(x)
        x = self.fc2(x)
        # result = F.sigmoid(x)
        result = x
        
        #raise 'Error'
        
        return result
    
    def init(self):
        torch.nn.init.constant(self.conv1.bias, val=0.0)
        torch.nn.init.constant(self.fc1.bias, val=0.0)
        torch.nn.init.constant(self.fc2.bias, val=0.0)

In [20]:
%autoreload 2

# criterion = nn.BCELoss()
criterion = nn.CrossEntropyLoss()
model = CnnMaxPoolingModule()
if(use_cuda):
    model.cuda()
    criterion.cuda()
trainer = ModuleTrainer(model)
trainer.set_optimizer(optim.Adam, lr=1e-3)
trainer.set_loss(criterion)
trainer.set_initializers([Uniform(module_filter="embedding*", a=-0.05, b=0.05), XavierUniform(module_filter="fc*"), XavierUniform(module_filter="conv*")])
trainer.set_metrics([CategoricalAccuracy()])

# trainer.summary((trn_tensor.size(0), labels_train_tensor.size(0)))
model

CnnMaxPoolingModule (
  (embedding): Embedding(5000, 32)
  (drop1): Dropout (p = 0.2)
  (conv1): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=(2,), groups=2)
  (fc1): Linear (16000 -> 100)
  (dropout): Dropout (p = 0.7)
  (fc2): Linear (100 -> 2)
)

In [22]:
trainer.fit(trn_tensor, labels_train_tensor, validation_data=(test_tensor, labels_test_tensor), 
            nb_epoch=2, batch_size=batch_size, shuffle=True)

Epoch 1/2: 392 batches [04:24, 18.87s/ batches, val_acc=80.01, val_loss=0.2810, acc=71.68, loss=0.5062]
Epoch 2/2: 392 batches [03:50, 17.43s/ batches, val_acc=89.87, val_loss=0.2576, acc=90.39, loss=0.2602]


In [23]:
trainer.fit(trn_tensor, labels_train_tensor, validation_data=(test_tensor, labels_test_tensor), 
            nb_epoch=4, batch_size=batch_size, shuffle=True)

Epoch 1/4: 392 batches [02:59,  7.73s/ batches, val_acc=77.54, val_loss=0.2922, acc=67.17, loss=0.5545]
Epoch 2/4: 392 batches [02:03,  7.54s/ batches, val_acc=89.12, val_loss=0.2622, acc=89.18, loss=0.2754]
Epoch 3/4: 392 batches [02:30,  9.34s/ batches, val_acc=90.64, val_loss=0.2588, acc=92.10, loss=0.2103]
Epoch 4/4: 392 batches [02:40,  9.55s/ batches, val_acc=91.03, val_loss=0.2911, acc=93.66, loss=0.1727]


### Pre-trained vectors

You may want to look at wordvectors.ipynb before moving on.

In this section, we replicate the previous CNN, but using pre-trained embeddings.

In [39]:
import torch
import re
from torchtext.vocab import load_word_vectors

wv_dict, wv_arr, wv_size = load_word_vectors('.', 'glove.6B', 50)

print('Loaded', len(wv_arr), 'words')

loading word vectors from ./glove.6B.50d.txt: 100%|██████████| 400000/400000 [00:10<00:00, 38319.17it/s]


Loaded 400000 words



The glove word ids and imdb word ids use different indexes. So we create a simple function that creates an embedding matrix using the indexes from imdb, and the embeddings from glove (where they exist).

In [146]:
def get_word(word):
    return wv_arr[wv_dict[word]]

def create_emb():
    num_dimensions_glove = wv_arr.size()[1]
    
    embedding = nn.Embedding(vocab_size, num_dimensions_glove)
    # If we can't find the word in glove, randomly initialize
    torch.nn.init.uniform(embedding.weight, a=-0.05, b=0.05)

    num_found, num_not_found = 0, 0
    
    for i in range(1,len(embedding.weight)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            embedding.weight.data[i] = get_word(word)
            num_found += 1
        else:
            num_not_found +=1

    # This is our "rare word" id - we want to randomly initialize
    torch.nn.init.uniform(embedding.weight.data[-1], a=-0.05, b=0.05)
    embedding.weight.requires_grad = False
    
    # This speeds up training. Can it be replaced by BatchNorm1d?
    embedding.weight.data /= 3
    
    print("Words found: {}, not found: {}".format(num_found, num_not_found))
    
    return embedding


We pass our embedding matrix to the Embedding constructor, and set it to non-trainable.

In [147]:
import torch.nn as nn
import torch.nn.functional as F

class CnnMaxPoolingModuleWithEmbedding(nn.Module):
    def __init__(self, embedding):
        super().__init__()
        num_dimensions = 32
        self.embedding = embedding
        self.drop1 = nn.Dropout(0.25)
        self.batchnorm = nn.BatchNorm1d(500)
        self.conv1 = nn.Conv1d(in_channels=embedding.weight.size()[1], out_channels=64, kernel_size=5, padding=2, groups=1)
        self.fc1 = nn.Linear(seq_len * num_dimensions, 100)
        self.dropout = nn.Dropout(0.7)
        self.fc2 = nn.Linear(100, 2)
        self.init()

    def forward(self, words_ids):
        x = self.embedding(words_ids)
        # x = self.batchnorm(x)
        x = x.permute(0, 2, 1)
        x = self.drop1(x)
        x = self.conv1(x)
        x = F.relu(x, True)
        x = self.drop1(x)
        x = F.max_pool1d(x, kernel_size=2)
        
        x = x.view(x.size(0), -1)
        
        x = self.fc1(x)
        x = F.relu(x, True)
        x = self.dropout(x)
        x = self.fc2(x)
        result = x
        
        return result
    
    def init(self):
        torch.nn.init.constant(self.conv1.bias, val=0.0)
        torch.nn.init.constant(self.fc1.bias, val=0.0)
        torch.nn.init.constant(self.fc2.bias, val=0.0)
        
    def parameters(self):
        p = filter(lambda p: p.requires_grad, nn.Module.parameters(self))
        return p

In [148]:
%autoreload 2

emb = create_emb()

# criterion = nn.BCELoss()
criterion = nn.CrossEntropyLoss()
model = CnnMaxPoolingModuleWithEmbedding(emb)
if(use_cuda):
    model.cuda()
    criterion.cuda()
trainer = ModuleTrainer(model)
trainer.set_optimizer(optim.Adam, lr=1e-3)
trainer.set_loss(criterion)
trainer.set_initializers([XavierUniform(module_filter="fc*"), XavierUniform(module_filter="conv*")])
trainer.set_metrics([CategoricalAccuracy()])

# trainer.summary((trn_tensor.size(0), labels_train_tensor.size(0)))

Words found: 4914, not found: 85


In [149]:
trainer.fit(trn_tensor, labels_train_tensor, validation_data=(test_tensor, labels_test_tensor), 
            nb_epoch=10, batch_size=batch_size, shuffle=True)

Epoch 1/10: 392 batches [05:23, 28.35s/ batches, loss=0.6532, acc=61.48, val_loss=0.5240, val_acc=69.12]
Epoch 2/10: 392 batches [04:36, 18.35s/ batches, loss=0.5188, acc=75.36, val_loss=0.4707, val_acc=77.29]
Epoch 3/10: 392 batches [04:55, 18.07s/ batches, loss=0.4685, acc=78.90, val_loss=0.4274, val_acc=80.36]
Epoch 4/10: 392 batches [03:10, 14.13s/ batches, loss=0.4317, acc=80.46, val_loss=0.3984, val_acc=81.39]
Epoch 5/10: 392 batches [02:58, 13.89s/ batches, loss=0.4150, acc=81.47, val_loss=0.3811, val_acc=82.45]
Epoch 6/10: 392 batches [03:30, 15.72s/ batches, loss=0.3967, acc=82.54, val_loss=0.3822, val_acc=83.00]
Epoch 7/10: 392 batches [03:31, 14.43s/ batches, loss=0.3758, acc=83.47, val_loss=0.3835, val_acc=83.19]
Epoch 8/10: 392 batches [03:06, 14.12s/ batches, loss=0.3543, acc=84.56, val_loss=0.3692, val_acc=84.13]
Epoch 9/10: 392 batches [02:55, 10.51s/ batches, loss=0.3453, acc=84.74, val_loss=0.3665, val_acc=84.38]
Epoch 10/10: 392 batches [03:03, 12.91s/ batches, loss=

We already have beaten our previous model! But let's fine-tune the embedding weights - especially since the words we couldn't find in glove just have random embeddings.

In [151]:
model.embedding.weight.requires_grad = True
trainer = ModuleTrainer(model)
trainer.set_optimizer(optim.Adam, lr=1e-4)
trainer.set_loss(criterion)
trainer.set_metrics([CategoricalAccuracy()])

In [152]:
trainer.fit(trn_tensor, labels_train_tensor, validation_data=(test_tensor, labels_test_tensor), 
            nb_epoch=1, batch_size=batch_size, shuffle=True)

Epoch 1/1: 392 batches [01:50,  6.58s/ batches, loss=0.3015, acc=87.19, val_loss=0.3479, val_acc=86.01]


### Multi-size CNN

This is an implementation of a multi-size CNN as shown in Ben Bowles' excellent blog post.

We create multiple conv layers of different sizes, and then concatenate them.

In [173]:
import torch.nn as nn
import torch.nn.functional as F

class CnnMaxPoolingModuleMultiSizeWithEmbedding(nn.Module):
    def __init__(self, embedding):
        super().__init__()
        num_dimensions = 32
        self.embedding = embedding
        self.drop1 = nn.Dropout(0.25)
        self.batchnorm = nn.BatchNorm1d(500)
        self.convs = [self.create_conv(embedding, fsz) for fsz in range (3, 6)]
        self.fc1 = nn.Linear(25000, 100)
        self.dropout = nn.Dropout(0.7)
        self.fc2 = nn.Linear(100, 2)
        self.init()
        
    def create_conv(self, embedding, fsz):
        return nn.Conv1d(in_channels=embedding.weight.size()[1], out_channels=64, kernel_size=5, padding=2, groups=1)
    
    def conv(self, c, x):
        x = c(x)
        x = F.relu(x, True)
        x = self.drop1(x)
        x = F.max_pool1d(x, kernel_size=2)
        return x

    def forward(self, words_ids):
        x = self.embedding(words_ids)
        x = x.permute(0, 2, 1)
        x = self.drop1(x)
        convs = [self.conv(conv, x) for conv in self.convs]
        
        torch.cat(convs, dim=1)
        
        x = x.view(x.size(0), -1)
        
        x = self.fc1(x)
        x = F.relu(x, True)
        x = self.dropout(x)
        x = self.fc2(x)
        result = x
        
        return result
    
    def init(self):
        torch.nn.init.constant(self.fc1.bias, val=0.0)
        torch.nn.init.constant(self.fc2.bias, val=0.0)
        for conv in self.convs:
            torch.nn.init.xavier_uniform(conv.weight.data, gain=1.0)
            torch.nn.init.constant(conv.bias, val=0.0)
        
    def parameters(self):
        p = filter(lambda p: p.requires_grad, nn.Module.parameters(self))
        return p

In [174]:
%autoreload 2

emb = create_emb()

criterion = nn.CrossEntropyLoss()
model = CnnMaxPoolingModuleMultiSizeWithEmbedding(emb)
model.embedding.weight.requires_grad = True
if(use_cuda):
    model.cuda()
    criterion.cuda()
trainer = ModuleTrainer(model)
trainer.set_optimizer(optim.Adam, lr=1e-3)
trainer.set_loss(criterion)
trainer.set_initializers([XavierUniform(module_filter="fc*")])
trainer.set_metrics([CategoricalAccuracy()])

Words found: 4914, not found: 85


In [175]:
trainer.fit(trn_tensor, labels_train_tensor, validation_data=(test_tensor, labels_test_tensor), 
            nb_epoch=10, batch_size=batch_size, shuffle=True)

Epoch 1/10: 392 batches [10:42, 73.36s/ batches, loss=0.6308, acc=61.92, val_loss=0.4106, val_acc=72.10]
Epoch 2/10: 392 batches [13:00, 70.60s/ batches, loss=0.4033, acc=81.30, val_loss=0.3117, val_acc=84.12]
Epoch 3/10: 392 batches [08:08, 28.02s/ batches, loss=0.2956, acc=87.68, val_loss=0.2967, val_acc=87.63]
Epoch 4/10: 392 batches [05:14, 33.21s/ batches, loss=0.2316, acc=90.32, val_loss=0.3037, val_acc=88.94]
Epoch 5/10: 392 batches [05:00, 23.83s/ batches, loss=0.1827, acc=92.86, val_loss=0.3306, val_acc=90.25]
Epoch 6/10: 392 batches [04:18, 25.20s/ batches, loss=0.1464, acc=94.25, val_loss=0.3538, val_acc=90.86]
Epoch 7/10: 392 batches [04:17, 21.28s/ batches, loss=0.1203, acc=95.12, val_loss=0.3973, val_acc=91.11]
Epoch 8/10: 392 batches [04:28, 27.56s/ batches, loss=0.0926, acc=96.44, val_loss=0.4414, val_acc=91.88]
Epoch 9/10: 392 batches [04:46, 29.97s/ batches, loss=0.0766, acc=97.03, val_loss=0.4905, val_acc=92.07]
Epoch 10/10: 392 batches [05:13, 32.26s/ batches, loss=

This is clearly over-fitting. But it does get the highest accuracy on validation set.

### LSTM

We haven't covered this bit yet!

In [317]:
import torch.nn as nn
import torch.nn.functional as F

class LstmEmbeddingModule(nn.Module):
    def __init__(self):
        super().__init__()
        num_dimensions = 32
        self.num_hidden = 100
        self.embedding = nn.Embedding(vocab_size, num_dimensions)
        self.drop1 = nn.Dropout(0.2)
        self.lstm1 = nn.LSTM(input_size=32, hidden_size=self.num_hidden, num_layers=1, batch_first=True)
        self.fc1 = nn.Linear(50000, 2)
        self.hidden = self.init_hidden(batch_size)
        self.init()

    def forward(self, words_ids):
        
        # We detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        # self.hidden = self.repackage_hidden(self.hidden)
        
        x = self.embedding(words_ids)
        x = self.drop1(x)
        #print('embd', x.size())
        
        self.hidden = self.init_hidden(x.size(0))
        
        #lenghts = [vocab_size for _ in range(x.size(0))]
        #x = torch.nn.utils.rnn.pack_padded_sequence(x, lenghts, batch_first=True)
        
        #print('pack', x.data.size())
        
        x, self.hidden = self.lstm1(x, self.hidden)
        
        #print('lstm', x.data.size())
        
        #x, _ = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)
        
        #print('unpk', x.size())
        
        # print(self.hidden)
        # TODO can we get rid of contiguous?
        x = x.contiguous().view(x.size(0), -1)
        #print('view', x.size())
        
        x = self.fc1(x)
        x = F.relu(x, True)
    
        return x
    
    def init(self):
        torch.nn.init.constant(self.fc1.bias, val=0.0)
        
    def init_hidden(self, batch_size):
        num_layers = 1
        weight = next(self.parameters()).data
        return (Variable(weight.new(num_layers, batch_size, self.num_hidden).zero_()),
                    Variable(weight.new(num_layers, batch_size, self.num_hidden).zero_()))

    def repackage_hidden(self, h):
        """Wraps hidden states in new Variables, to detach them from their history."""
        if type(h) == Variable:
            return Variable(h.data)
        else:
            return tuple(self.repackage_hidden(v) for v in h)

In [318]:
%autoreload 2

criterion = nn.CrossEntropyLoss()
model = LstmEmbeddingModule()
if(use_cuda):
    model.cuda()
    criterion.cuda()
trainer = ModuleTrainer(model)
trainer.set_optimizer(optim.Adam, lr=1e-3)
trainer.set_loss(criterion)
# TODO init LSTM
trainer.set_initializers([Uniform(module_filter="embedding*", a=-0.05, b=0.05), XavierUniform(module_filter="fc*"), XavierUniform(module_filter="conv*")])
trainer.set_metrics([CategoricalAccuracy()])

In [319]:
# TODO figure out how to do this in PyTorch
trainer.fit(trn_tensor, labels_train_tensor, validation_data=(test_tensor, labels_test_tensor), 
            nb_epoch=5, batch_size=batch_size, shuffle=True)

Epoch 1/5: 392 batches [40:42, 66.02s/ batches, loss=0.6943, acc=50.00, val_loss=0.6931, val_acc=50.00]
Epoch 2/5:   1%|▏         | 5/391 [00:10<12:52,  2.00s/ batches, loss=0.6931, acc=51.95]


KeyboardInterrupt: 